In [17]:
import pandas as pd

In [27]:
# Carico il file CSV
df = pd.read_csv('../420mood/FMP/Coppa/Miti/events.csv')

# Mostro le prime 10 righe per una prima esplorazione
df.head(10)


,Nome,Posizione,Durata,Data,Evento,Portiere,Quartetto,Quartetto.1,Quartetto.2,Quartetto.3,Quartetto.4,Chi,Esito,Field Position,Piede,Squadra
0,Inizio avversario (1),0:19:01.000,0:00:10.000,05/01/2025,Inizio avversario,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,NaN
1,Tiro (1),0:19:17.500,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,digao,Ribattuto,2.0,Dx,Loro
2,Angolo (1),0:19:33.366,0:00:08.000,05/01/2025,Angolo,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,Loro
3,Tiro (2),0:19:40.400,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,NaN,Parata,2.0,Dx,Loro
4,Palla recuperata (1),0:20:17.000,0:00:10.000,05/01/2025,Palla recuperata,giovi,ricky,nic,digao,diego jr,NaN,digao,Fuori,0.0,NaN,NaN
5,Laterale (1),0:20:26.000,0:00:07.000,05/01/2025,Laterale,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,0.0,NaN,Loro
6,Laterale (2),0:20:58.000,0:00:07.000,05/01/2025,Laterale,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,0.0,NaN,Noi
7,Laterale (3),0:21:38.633,0:00:07.000,05/01/2025,Laterale,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,6.0,NaN,Noi
8,Tiro (3),0:21:45.466,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,nic,Ribattuto,5.0,Dx,Noi
9,Laterale (4),0:21:59.333,0:00:07.000,05/01/2025,Laterale,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,1.0,NaN,Noi


In [31]:
import numpy as np
import pandas as pd

def to_seconds(time_str):
    """Converte una stringa 'h:mm:ss.xxx' in secondi float."""
    if pd.isna(time_str):
        return np.nan
    t = time_str.split(':')
    if len(t) == 3:
        h, m, s = t
        return int(h)*3600 + int(m)*60 + float(s)
    else:
        return np.nan

def format_mmss(s):
    if pd.isna(s):
        return ''
    m = int(s // 60)
    s2 = int(round(s % 60))
    return f"{m:02}:{s2:02}"

def calcola_tempo_effettivo(df):
    df_eff = df.copy()
    df_eff['Posizione_sec'] = df_eff['Posizione'].apply(to_seconds)

    # Trova indici chiave
    idx_start = 0
    idx_fine_primo = df_eff[df_eff['Evento'] == 'Fine primo tempo'].index[0]
    idx_start_secondo = idx_fine_primo + 1
    idx_fine_partita = df_eff[df_eff['Evento'] == 'Fine partita'].index[0]

    # Tempi reali (in secondi)
    t_start = df_eff.loc[idx_start, 'Posizione_sec']
    t_fine_primo = df_eff.loc[idx_fine_primo, 'Posizione_sec']
    t_start_secondo = df_eff.loc[idx_start_secondo, 'Posizione_sec']
    t_fine_partita = df_eff.loc[idx_fine_partita, 'Posizione_sec']

    tempo_effettivo_sec = []

    for i, row in df_eff.iterrows():
        t = row['Posizione_sec']
        if i <= idx_fine_primo:
            # Primo tempo
            perc = (t - t_start) / (t_fine_primo - t_start) if (t_fine_primo - t_start) != 0 else 0
            eff = perc * 20 * 60
        elif i >= idx_start_secondo:
            # Secondo tempo
            perc = (t - t_start_secondo) / (t_fine_partita - t_start_secondo) if (t_fine_partita - t_start_secondo) != 0 else 0
            eff = 20 * 60 + perc * 20 * 60
        else:
            eff = np.nan  # Tra fine primo tempo e inizio secondo
        tempo_effettivo_sec.append(eff)

    df_eff['tempoEffettivo_sec'] = tempo_effettivo_sec
    df_eff['tempoEffettivo'] = df_eff['tempoEffettivo_sec'].apply(format_mmss)

    return df_eff


In [32]:
df = calcola_tempo_effettivo(df)

In [33]:
df

,Nome,Posizione,Durata,Data,Evento,Portiere,Quartetto,Quartetto.1,Quartetto.2,Quartetto.3,Quartetto.4,Chi,Esito,Field Position,Piede,Squadra,Posizione_sec,tempoEffettivo_sec,tempoEffettivo
0,Inizio avversario (1),0:19:01.000,0:00:10.000,05/01/2025,Inizio avversario,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,NaN,1141.000,0.000000,00:00
1,Tiro (1),0:19:17.500,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,digao,Ribattuto,2.0,Dx,Loro,1157.500,8.061781,00:08
2,Angolo (1),0:19:33.366,0:00:08.000,05/01/2025,Angolo,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,Loro,1173.366,15.813794,00:16
3,Tiro (2),0:19:40.400,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,NaN,Parata,2.0,Dx,Loro,1180.400,19.250556,00:19
4,Palla recuperata (1),0:20:17.000,0:00:10.000,05/01/2025,Palla recuperata,giovi,ricky,nic,digao,diego jr,NaN,digao,Fuori,0.0,NaN,NaN,1217.000,37.133052,00:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,5v4 (12),1:53:10.400,0:00:10.000,05/01/2025,5v4,giovi,fabri,ricky,diego jr,digao,NaN,NaN,NaN,NaN,NaN,Loro,6790.400,2384.232619,39:44
330,Laterale (78),1:53:26.000,0:00:07.000,05/01/2025,Laterale,giovi,fabri,ricky,diego jr,digao,NaN,NaN,NaN,0.0,NaN,Noi,6806.000,2391.474306,39:51
331,Palla persa (12),1:53:32.266,0:00:10.000,05/01/2025,Palla persa,giovi,fabri,ricky,diego jr,digao,NaN,diego jr,NaN,0.0,NaN,NaN,6812.266,2394.383050,39:54
332,Palla recuperata (37),1:53:33.766,0:00:10.000,05/01/2025,Palla recuperata,giovi,fabri,ricky,diego jr,digao,NaN,digao,NaN,NaN,NaN,NaN,6813.766,2395.079366,39:55


In [22]:

# Funzione conversione secondi
def to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    t = time_str.split(':')
    if len(t) == 3:
        h, m, s = t
        return int(h)*3600 + int(m)*60 + float(s)
    else:
        return np.nan

df['tempoReale_sec'] = df['Posizione'].apply(to_seconds)

# Trova indici per segmentare tempi
idx_inizio = 0
idx_fine_primo = df[df['Evento'] == 'Fine primo tempo'].index[0]
idx_inizio_secondo = idx_fine_primo + 1
idx_fine_partita = df[df['Evento'] == 'Fine partita'].index[0]

# Durate
durata_primo = df.loc[idx_fine_primo, 'tempoReale_sec'] - df.loc[idx_inizio, 'tempoReale_sec']
durata_secondo = df.loc[idx_fine_partita, 'tempoReale_sec'] - df.loc[idx_inizio_secondo, 'tempoReale_sec']
durata_totale = durata_primo + durata_secondo

# Tabella riassuntiva durate
dati_durate = pd.DataFrame({
    "Periodo": ["Primo tempo", "Secondo tempo", "Totale partita"],
    "Durata_minuti": [round(durata_primo/60), round(durata_secondo/60), round(durata_totale/60)]
})



In [23]:
dati_durate

,Periodo,Durata_minuti
0,Primo tempo,41
1,Secondo tempo,43
2,Totale partita,84


In [24]:
df

,Nome,Posizione,Durata,Data,Evento,Portiere,Quartetto,Quartetto.1,Quartetto.2,Quartetto.3,Quartetto.4,Chi,Esito,Field Position,Piede,Squadra,Posizione_sec,tempoEffettivo_sec,tempoEffettivo,tempoReale_sec
0,Inizio avversario (1),0:19:01.000,0:00:10.000,05/01/2025,Inizio avversario,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,NaN,1141.000,0.000000,00:00,1141.000
1,Tiro (1),0:19:17.500,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,digao,Ribattuto,2.0,Dx,Loro,1157.500,8.061781,00:08,1157.500
2,Angolo (1),0:19:33.366,0:00:08.000,05/01/2025,Angolo,giovi,ricky,nic,digao,diego jr,NaN,NaN,NaN,NaN,NaN,Loro,1173.366,15.813794,00:16,1173.366
3,Tiro (2),0:19:40.400,0:00:07.000,05/01/2025,Tiro,giovi,ricky,nic,digao,diego jr,NaN,NaN,Parata,2.0,Dx,Loro,1180.400,19.250556,00:19,1180.400
4,Palla recuperata (1),0:20:17.000,0:00:10.000,05/01/2025,Palla recuperata,giovi,ricky,nic,digao,diego jr,NaN,digao,Fuori,0.0,NaN,NaN,1217.000,37.133052,00:37,1217.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,5v4 (12),1:53:10.400,0:00:10.000,05/01/2025,5v4,giovi,fabri,ricky,diego jr,digao,NaN,NaN,NaN,NaN,NaN,Loro,6790.400,1124.232619,18:44,6790.400
330,Laterale (78),1:53:26.000,0:00:07.000,05/01/2025,Laterale,giovi,fabri,ricky,diego jr,digao,NaN,NaN,NaN,0.0,NaN,Noi,6806.000,1131.474306,18:51,6806.000
331,Palla persa (12),1:53:32.266,0:00:10.000,05/01/2025,Palla persa,giovi,fabri,ricky,diego jr,digao,NaN,diego jr,NaN,0.0,NaN,NaN,6812.266,1134.383050,18:54,6812.266
332,Palla recuperata (37),1:53:33.766,0:00:10.000,05/01/2025,Palla recuperata,giovi,fabri,ricky,diego jr,digao,NaN,digao,NaN,NaN,NaN,NaN,6813.766,1135.079366,18:55,6813.766


In [25]:
df['tempoReale_sec'] = df['Posizione'].apply(to_seconds)

def get_giocatori_in_campo(row):
    giocatori = [row['Portiere']]
    for col in ['Quartetto', 'Quartetto.1', 'Quartetto.2', 'Quartetto.3', 'Quartetto.4']:
        if pd.notna(row.get(col)):
            giocatori.append(row[col])
    giocatori = [g for g in giocatori if pd.notna(g) and str(g).strip() != '']
    if len(giocatori) == 5:
        return tuple(sorted(giocatori[1:]))
    else:
        return tuple(sorted(giocatori))

df['quartetto'] = df.apply(get_giocatori_in_campo, axis=1)
df['delta_reale'] = df['tempoReale_sec'].shift(-1) - df['tempoReale_sec']
df['delta_reale'] = df['delta_reale'].fillna(0)

# Funzione per tabella minuti e percentuale
def tabella_minuti_percentuali(df_sub, quartetti=True, coppie=False, singoli=False, durata_rif=None):
    from itertools import combinations
    from collections import defaultdict

    # Determina durata di riferimento
    if durata_rif is None:
        durata_rif = df_sub['tempoReale_sec'].sum() if len(df_sub) == 0 else df_sub['delta_reale'].sum()
    acc = defaultdict(float)

    for idx, row in df_sub.iterrows():
        giocatori = list(row['quartetto'])
        delta = row['delta_reale']
        if quartetti and len(giocatori) >= 4:
            key = tuple(sorted(giocatori))
            acc[key] += delta
        if coppie:
            for c in combinations(sorted(giocatori), 2):
                acc[c] += delta
        if singoli:
            for g in giocatori:
                acc[g] += delta
    # Prepara dataframe
    if quartetti:
        df_out = pd.DataFrame([
            {"Quartetto_o_Quintetto": q, "Minuti_giocati": t/60, "Percentuale": 100*t/durata_rif} for q, t in acc.items()
        ]).sort_values(by="Minuti_giocati", ascending=False)
    elif coppie:
        df_out = pd.DataFrame([
            {"Coppia": c, "Minuti_giocati": t/60, "Percentuale": 100*t/durata_rif} for c, t in acc.items()
        ]).sort_values(by="Minuti_giocati", ascending=False)
    elif singoli:
        df_out = pd.DataFrame([
            {"Giocatore": g, "Minuti_giocati": t/60, "Percentuale": 100*t/durata_rif} for g, t in acc.items()
        ]).sort_values(by="Minuti_giocati", ascending=False)
    else:
        df_out = pd.DataFrame()
    return df_out

# Indici per tempi
idx_inizio = 0
idx_fine_primo = df[df['Evento'] == 'Fine primo tempo'].index[0]
idx_inizio_secondo = idx_fine_primo + 1
idx_fine_partita = df[df['Evento'] == 'Fine partita'].index[0]

# Durate
durata_primo = df.loc[idx_fine_primo, 'tempoReale_sec'] - df.loc[idx_inizio, 'tempoReale_sec']
durata_secondo = df.loc[idx_fine_partita, 'tempoReale_sec'] - df.loc[idx_inizio_secondo, 'tempoReale_sec']
durata_totale = durata_primo + durata_secondo

# 1. Quartetti/quintetti
df_quartetti_tot = tabella_minuti_percentuali(df, quartetti=True, durata_rif=durata_totale)
df_quartetti_1t = tabella_minuti_percentuali(df.iloc[idx_inizio:idx_fine_primo+1], quartetti=True, durata_rif=durata_primo)
df_quartetti_2t = tabella_minuti_percentuali(df.iloc[idx_inizio_secondo:idx_fine_partita+1], quartetti=True, durata_rif=durata_secondo)

# 2. Singoli
df_singoli_tot = tabella_minuti_percentuali(df, quartetti=False, singoli=True, durata_rif=durata_totale)
df_singoli_1t = tabella_minuti_percentuali(df.iloc[idx_inizio:idx_fine_primo+1], quartetti=False, singoli=True, durata_rif=durata_primo)
df_singoli_2t = tabella_minuti_percentuali(df.iloc[idx_inizio_secondo:idx_fine_partita+1], quartetti=False, singoli=True, durata_rif=durata_secondo)

# 3. Coppie
df_coppie_tot = tabella_minuti_percentuali(df, quartetti=False, coppie=True, durata_rif=durata_totale)
df_coppie_1t = tabella_minuti_percentuali(df.iloc[idx_inizio:idx_fine_primo+1], quartetti=False, coppie=True, durata_rif=durata_primo)
df_coppie_2t = tabella_minuti_percentuali(df.iloc[idx_inizio_secondo:idx_fine_partita+1], quartetti=False, coppie=True, durata_rif=durata_secondo)


In [26]:
# Quartetti/Quintetti
display(df_quartetti_tot)   # Totale partita
display(df_quartetti_1t)    # Primo tempo
display(df_quartetti_2t)    # Secondo tempo

# Singoli
display(df_singoli_tot)
display(df_singoli_1t)
display(df_singoli_2t)

# Coppie
display(df_coppie_tot)
display(df_coppie_1t)
display(df_coppie_2t)


,Quartetto_o_Quintetto,Minuti_giocati,Percentuale
2,"(deba, erick, fabri, mattia)",32.547217,38.738493
0,"(diego jr, digao, nic, ricky)",24.785567,29.500387
3,"(diego jr, fabri, nic, ricky)",19.754983,23.512864
11,"(digao, fabri, nico estratti, ricky)",6.448900,7.675638
4,"(deba, diego jr, digao, nic)",2.642233,3.144851
6,"(diego jr, fabri, nic, nico estratti)",1.576667,1.876587
5,"(diego jr, digao, nic, nico estratti)",1.109450,1.320495
12,"(deba, diego jr, erick, fabri)",1.063333,1.265605
14,"(deba, diego jr, fabri, nico estratti)",0.978333,1.164436
9,"(diego jr, digao, fabri, mattia)",0.819450,0.975329


,Quartetto_o_Quintetto,Minuti_giocati,Percentuale
2,"(deba, erick, fabri, mattia)",17.281100,42.217104
0,"(diego jr, digao, nic, ricky)",14.623883,35.725619
3,"(diego jr, fabri, nic, ricky)",13.758883,33.612456
4,"(deba, diego jr, digao, nic)",2.642233,6.454881
6,"(diego jr, fabri, nic, nico estratti)",1.576667,3.851740
5,"(diego jr, digao, nic, nico estratti)",1.109450,2.710346
1,"(diego jr, nic, nico estratti, ricky)",0.646667,1.579783


,Quartetto_o_Quintetto,Minuti_giocati,Percentuale
0,"(deba, erick, fabri, mattia)",15.266117,35.433474
1,"(diego jr, digao, nic, ricky)",10.161683,23.585811
7,"(digao, fabri, nico estratti, ricky)",6.448900,14.968242
6,"(diego jr, fabri, nic, ricky)",5.996100,13.917269
8,"(deba, diego jr, erick, fabri)",1.063333,2.468054
10,"(deba, diego jr, fabri, nico estratti)",0.978333,2.270764
4,"(diego jr, digao, fabri, mattia)",0.819450,1.901987
3,"(deba, diego jr, digao, mattia)",0.743317,1.725278
11,"(diego jr, digao, fabri, ricky)",0.651100,1.511238
2,"(erick, fabri, mattia, nico estratti)",0.360550,0.836856


,Giocatore,Minuti_giocati,Percentuale
7,fabri,64.795533,77.121228
0,diego jr,55.366100,65.898086
3,ricky,52.570550,62.570754
2,nic,50.827233,60.495816
5,deba,38.257767,45.535329
1,digao,37.511683,44.647323
8,mattia,34.470533,41.027672
6,erick,34.254433,40.770464
4,nico estratti,11.120567,13.235970
9,giovi,0.283333,0.337230


,Giocatore,Minuti_giocati,Percentuale
0,diego jr,34.357783,83.934825
2,nic,34.357783,83.934825
7,fabri,32.616650,79.681299
3,ricky,29.029433,70.917858
5,deba,19.923333,48.671984
1,digao,18.375567,44.890846
6,erick,17.281100,42.217104
8,mattia,17.281100,42.217104
4,nico estratti,3.332783,8.141869


,Giocatore,Minuti_giocati,Percentuale
2,fabri,32.178883,74.688911
7,ricky,23.541117,54.640192
4,diego jr,21.008317,48.761428
5,digao,19.136117,44.415951
0,deba,18.334433,42.555201
3,mattia,17.189433,39.897595
1,erick,16.973333,39.396015
6,nic,16.469450,38.226475
8,nico estratti,7.787783,18.075862
9,giovi,0.283333,0.657632


,Coppia,Minuti_giocati,Percentuale
1,"(diego jr, nic)",50.827233,60.495816
2,"(diego jr, ricky)",46.121650,54.895115
5,"(nic, ricky)",45.187217,53.782930
10,"(deba, fabri)",34.872217,41.505765
12,"(erick, fabri)",34.254433,40.770464
9,"(deba, erick)",33.893883,40.341329
14,"(fabri, mattia)",33.727217,40.142958
11,"(deba, mattia)",33.290533,39.623207
13,"(erick, mattia)",32.907767,39.167628
4,"(digao, ricky)",31.885567,37.950981


,Coppia,Minuti_giocati,Percentuale
1,"(diego jr, nic)",34.357783,83.934825
2,"(diego jr, ricky)",29.029433,70.917858
5,"(nic, ricky)",29.029433,70.917858
0,"(diego jr, digao)",18.375567,44.890846
3,"(digao, nic)",18.375567,44.890846
10,"(deba, fabri)",17.281100,42.217104
14,"(fabri, mattia)",17.281100,42.217104
13,"(erick, mattia)",17.281100,42.217104
12,"(erick, fabri)",17.281100,42.217104
11,"(deba, mattia)",17.281100,42.217104


,Coppia,Minuti_giocati,Percentuale
1,"(deba, fabri)",17.591117,40.829924
10,"(digao, ricky)",17.261683,40.065291
8,"(diego jr, ricky)",17.092217,39.671950
3,"(erick, fabri)",16.973333,39.396015
0,"(deba, erick)",16.612783,38.559160
7,"(diego jr, nic)",16.469450,38.226475
5,"(fabri, mattia)",16.446117,38.172317
11,"(nic, ricky)",16.157783,37.503080
2,"(deba, mattia)",16.009433,37.158752
4,"(erick, mattia)",15.626667,36.270330


In [35]:
# Conta gol fatti e subiti
gol_fatti = df[(df['Evento'] == 'Gol') & (df['Squadra'] != 'Loro')]
gol_subiti = df[(df['Evento'] == 'Gol') & (df['Squadra'] == 'Loro')]

# Conta tiri fatti e subiti
tiri_fatti = df[(df['Evento'] == 'Tiro') & (df['Squadra'] != 'Loro')]
tiri_subiti = df[(df['Evento'] == 'Tiro') & (df['Squadra'] == 'Loro')]

# Posizioni dei tiri
tiri_fatti_pos = tiri_fatti[['Posizione', 'Field Position']]
tiri_subiti_pos = tiri_subiti[['Posizione', 'Field Position']]

# Riepilogo generale
riepilogo_eventi = pd.DataFrame({
    "Tipo": ["Gol fatti", "Gol subiti", "Tiri fatti", "Tiri subiti"],
    "Numero": [len(gol_fatti), len(gol_subiti), len(tiri_fatti), len(tiri_subiti)]
})




In [36]:
riepilogo_eventi

,Tipo,Numero
0,Gol fatti,2
1,Gol subiti,1
2,Tiri fatti,52
3,Tiri subiti,36


In [38]:
# Funzione per filtrare gli eventi per tempo
def filtra_per_tempo(df, tempo='primo'):
    idx_fine_primo = df[df['Evento'] == 'Fine primo tempo'].index[0]
    idx_fine_partita = df[df['Evento'] == 'Fine partita'].index[0]

    if tempo == 'primo':
        return df.loc[:idx_fine_primo]
    elif tempo == 'secondo':
        return df.loc[idx_fine_primo+1:idx_fine_partita]
    else:
        return df

# Applichiamo la funzione e analizziamo tiri e gol
df_primo = filtra_per_tempo(df, 'primo')
df_secondo = filtra_per_tempo(df, 'secondo')



In [39]:
# Funzione generica per contare eventi specifici fatti/subiti
def conta_eventi(df_parziale, evento):
    fatti = len(df_parziale[(df_parziale['Evento'] == evento) & (df_parziale['Squadra'] != 'Loro')])
    subiti = len(df_parziale[(df_parziale['Evento'] == evento) & (df_parziale['Squadra'] == 'Loro')])
    return fatti, subiti

# Lista di eventi da analizzare
eventi_da_contare = ['Gol', 'Tiri', 'Parate', 'Angolo', 'Laterale', 'Fallo', 'Ammonizione', 'Espulsione', 'Timeout', '3v2', '2v1']

# Dizionario per raccogliere i dati
dati_eventi = {
    "Evento": [],
    "Fatti primo tempo": [],
    "Subiti primo tempo": [],
    "Fatti secondo tempo": [],
    "Subiti secondo tempo": [],
    "Fatti totali": [],
    "Subiti totali": []
}

# Compila i dati
for evento in eventi_da_contare:
    fatti_primo, subiti_primo = conta_eventi(df_primo, evento)
    fatti_secondo, subiti_secondo = conta_eventi(df_secondo, evento)
    dati_eventi["Evento"].append(evento)
    dati_eventi["Fatti primo tempo"].append(fatti_primo)
    dati_eventi["Subiti primo tempo"].append(subiti_primo)
    dati_eventi["Fatti secondo tempo"].append(fatti_secondo)
    dati_eventi["Subiti secondo tempo"].append(subiti_secondo)
    dati_eventi["Fatti totali"].append(fatti_primo + fatti_secondo)
    dati_eventi["Subiti totali"].append(subiti_primo + subiti_secondo)

df_eventi_divisi = pd.DataFrame(dati_eventi)



In [40]:
df_eventi_divisi

,Evento,Fatti primo tempo,Subiti primo tempo,Fatti secondo tempo,Subiti secondo tempo,Fatti totali,Subiti totali
0,Gol,0,0,2,1,2,1
1,Tiri,0,0,0,0,0,0
2,Parate,0,0,0,0,0,0
3,Angolo,9,7,10,4,19,11
4,Laterale,22,18,18,20,40,38
5,Fallo,5,5,3,5,8,10
6,Ammonizione,2,1,2,2,4,3
7,Espulsione,0,0,0,1,0,1
8,Timeout,0,1,1,0,1,1
9,3v2,1,1,0,0,1,1
